In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from utils import data, plot, evaluate
from models import GRU, LSTM

# Set device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Load Data

In [ ]:
data_dir = 'data/CMAPSS'
subset = 1
train, test, header, n_meta, n_sensors, n_settings = data.load_cmapss_subset(
    data_dir, subset)

params = {
    'drop_sensors': [1, 5, 6, 10, 16, 18, 19],
    'include_settings': None,
    'smooth': 0,
    'lag': 20,
    'clip': 125,
    'cv_folds': None,
    'return_cols': True,
    'reshape_2d': True
}
X, y, _, cols = data.build_dataset(train, **params)
X_test, y_test, _, cols = data.build_dataset(test, **params, test=True)
print(X.shape, y.shape)

batch_size = 2048

train_data = TensorDataset(
    torch.from_numpy(X),
    torch.from_numpy(y))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

test_data = TensorDataset(
    torch.from_numpy(X_test),
    torch.from_numpy(y_test))
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size, drop_last=False)

In [ ]:
def train(device, train_loader, learning_rate, hidden_dim=256, epochs=5, model_type="GRU"):
    
    # Setting common hyperparameters
    input_dim = next(iter(train_loader))[0].shape[2]
    output_dim = 1
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRU(device, input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTM(device, input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    # Start training loop
    for epoch in range(1, epochs+1):
        start_time = time.perf_counter()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.perf_counter()
        print("Epoch {}/{} Done, Total Loss: {}".format(epoch, epochs, avg_loss/len(train_loader)))
        print("Time Elapsed for Epoch: {} seconds".format(str(current_time-start_time)))
        epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model


def evaluate_network(model, test_x, test_y):
    model.eval()
    start_time = time.perf_counter()
    inp = torch.from_numpy(np.array(test_x))
    labs = torch.from_numpy(np.array(test_y))
    h = model.init_hidden(inp.shape[0])
    out, h = model(inp.to(device).float(), h)
    outputs = out.cpu().detach().numpy().reshape(-1)
    metrics = evaluate.metrics(test_y, outputs)
    print("Evaluation Time: {}".format(str(time.perf_counter()-start_time)))
    return outputs, metrics

In [ ]:
print('GRU Model\n------------------')
lr = 0.001
gru_model = train(device, train_loader, lr, epochs=1, model_type="GRU")
print('---')
gru_outputs, metrics = evaluate_network(gru_model, X_test, y_test)
print('Test Set - MAPE={:.3f} - MAE={:.3f} - MSE={:.3f} - RMSE={:.3f} - R2={:.3f}'.format(*metrics))

In [ ]:
print('LSTM Model\n------------------')
lr = 0.001
lstm_model = train(device, train_loader, lr, epochs=1, model_type="LSTM")
print('---')
gru_outputs, metrics = evaluate_network(lstm_model, X_test, y_test)
print('Test Set - MAPE={:.3f} - MAE={:.3f} - MSE={:.3f} - RMSE={:.3f} - R2={:.3f}'.format(*metrics))